#
# DEIM 2024 チュートリアル **大規模言語モデルに基づく検索モデル** \[TU-C-2\]

## 概要
近年の大規模言語モデルに基づく検索モデルを用いた検索実験のデモ

DPRの学習と評価のデモを通して、よく用いられているフレームワークやツールを示します．

ここでは，**NFCorpus**[1]というデータセットを用いて，DPRというBERTベースの密検索モデルを訓練します．
1. ファインチューニングする前に、DPRのnfcorpusでの性能を評価します．
1. nfcorpusの訓練セットでDPRを訓練します．
1. 2.で得られたDPRを再びnfcorpusで評価し，DPRのin-domainの検索性能を確認します．

[1]: Boteva et al. A Full-Text Learning to Rank Dataset for Medical Information Retrieval. ECIR, 2016.

## 始める前に
**ランタイムのタイプがGPU（e.g. T4 GPU）になっていることを確認してください！**

確認方法
- colab上部のナビゲーションバーから「ランタイム」> 「ランタイムのタイプを変更」
- 「ハードウェア アクセラレータ」の指定がGPUになっていれば閉じる。
- なっていなければ選択できるGPUを指定して「保存」を押す

## 依存関係のインストール

今回は、DPRのファインチューニングにTevatronという大規模言語モデルの訓練ツールキットを使います。

そのため、まずはTevatronと、依存パッケージをインストールします。

### **Tevatron**

大規模言語モデルに基づく検索モデルの学習や評価に焦点を当てたツールキット・pythonフレームワークです。

情報検索系のフレームワークの中では大規模言語モデル系の検索モデルの学習に強いという特徴があり、Tevatronを用いて検索モデルの訓練を行った研究も多数存在しています。

- Gao et al. Tevatron: An efficient and flexible toolkit for dense retrieval. SIGIR, 2023.
- Zhao et al. Dense Text Retrieval based on Pretrained Language Models: A Survey, arXiv, 2022.
- Zhuang eet al. CharacterBERT and Self-Teaching for Improving the Robustness of Dense Retrievers on Queries with Typos. SIGIR, 2022.
- Lin et al. Aggretriever: A Simple Approach to Aggregate Textual Representations for Robust Dense Passage Retrieval, TACL, 2023
- etc ...



### その他検索実験系ツール
- Pyserini https://github.com/castorini/pyserini/
  - 検索ツールキットで，「文の実験の再現を２クリックでできる」というように、簡単に論文の結果を再現するというところに焦点を当てています．
  - 事前にエンコードされたインデックスが多数ホストされており、手軽に実験を始めることができます。
  - 実装されている検索モデルの数が多く、exampleも充実しています。
  - 今回紹介するツールの中では最もgithubスター数が多いです。
- Pyterrier https://github.com/terrier-org/pyterrier
  - 拡張性・柔軟性が高い情報検索フレームワークです。
  - パイプライン機能が特徴的で、宣言的に検索実験のコードが書けるようになっています。
  - 今回のチュートリアルで紹介されたような検索モデルも一部pluginとして利用できるようになっています。
  - ちょっと捻ったような検索実験をするときは、フレームワークとしてとても有用だと思います。
    - パイプラインの拡張性が高くなるように設計されているため、新しい検索モデルも統合しやすいです。昔からあるモデルはpyterrierに組み込まれているため、pyterrierだけで新旧の検索モデルを包括的に実験ができます。
  - 後述するir-datasetsとの相性も良いです。

In [ ]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
info: syncing channel updates for 'stable-x86_64-unknown-linux-gnu'
info: latest update on 2024-02-08, rust version 1.76.0 (07dca489a 2024-02-04)
info: downloading component 'cargo'
info: downloading component 'clippy'
info: downloading component 'rust-docs'
info: downloading component 'rust-std'
info: downloading component 'rustc'
info: downloading component 'rustfmt'
info: installing component 'cargo'
info: installing component 'clippy'
info: installing component 'rust-docs'
 14.7 MiB /  14.7 MiB (100 %)   1.3 MiB/s in 10s ETA:  0s
info: installing component 'rust-std'
 23.9 MiB /  23.9 MiB (100 %)  10.0 MiB/s in  2s ETA:  0s
info: installing component 'rustc'
 62.3 MiB /  62.3 MiB (100 %)   9.6 MiB/s in  6s ETA:  0s
info: installing component 'rustfmt'
info: default toolchain set to 'stable-x86_64-unknown-linux-gnu'

  stable-x86_64-unknown-linux-gnu installed - rustc 1.7

In [ ]:
!pip install git+https://github.com/texttron/tevatron
!pip install git+https://github.com/luyug/GradCache
!pip install torch==1.11.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!PATH="$HOME/.cargo/bin:$PATH" pip install -U faiss-cpu==1.7.2 transformers==4.16.0 datasets==1.17.0

  Cloning https://github.com/texttron/tevatron to /tmp/pip-req-build-mla0j9xw
  Running command git clone --filter=blob:none --quiet https://github.com/texttron/tevatron /tmp/pip-req-build-mla0j9xw
  Resolved https://github.com/texttron/tevatron to commit 2e5d00ee21d5a7db0bd2ea1463c9150a572106d4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
  Created wheel for tevatron: filename=tevatron-0.0.1-py3-none-any.whl size=38500 sha256=1723654e70eeca3665d2d3fc85f24ac95d3adc1fb064fa0d96629de87ca78120
  Stored in directory: /tmp/pip-ephem-wheel-cache-43t9qihd/wheels/bc/1e/c8/3bc95e9e1ca1ebe94b78ca794b9a19de36adf4c6faf86b2346
Successfully built tevatron
  Cloning https://github.com/luyug/GradCache to /tmp/pip-req-build-3micz9i2
  Running command git clone -

こちらは評価に使うパッケージと、評価スクリプトのコードを使いたいので、Tevatronのソースコードをcloneします。

In [ ]:
!pip install pyserini
!git clone https://github.com/texttron/tevatron.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 89.2 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.9 MB/s eta 0:00:00
  Created 

## DPRの現時点の検索性能

今回はmsmarco-passageでfine-tuningされたDPRを、nfcorpusでファインチューニングしていきます。

その前に、今の時点ではどのくらいの性能が出ているのか見てみましょう。

### Tevatronのスクリプトを用いて、nfcorpusで評価

nfcorpusはBEIRというベンチマークに含まれているデータセットの一つです。

Tevatronには検索モデルをBEIRで簡単に評価できるようなスクリプトがあるので使わせてもらいましょう。
BEIRの論文ではRecall@100とnDCG@10で評価しているため、このスクリプトもそれに倣っています。
そのため、今回用いる評価指標もRecall@100、nDCG@10です。

In [ ]:
%env BASE_IR_MODEL=k-ush/tevatron_dpr

env: BASE_IR_MODEL=k-ush/tevatron_dpr


In [ ]:
!mkdir beir_embedding_k-ush
!bash tevatron/scripts/eval_beir.sh $BASE_IR_MODEL nfcorpus

2024-03-01 17:28:22.817887: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 17:28:22.817931: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 17:28:22.819366: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 17:28:22.826612: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 17:28:24.232177: W tensorflow/comp

評価が終わりました！

評価結果を見やすく表形式で表示してみましょう。

In [ ]:
import os
import pandas as pd
from typing import Dict

def read_eval_result(result_filename: str) -> Dict[str, float]:
  recall_100 = None
  ndcg_cut_10 = None
  with open(result_filename) as f:
    for line in f:
      line = line.strip()
      if "recall_100" in line:
          recall_100 = float(line.split("\t")[-1])

      if "ndcg_cut_10" in line:
          ndcg_cut_10 = float(line.split("\t")[-1])
  return {
      "Recall@100": recall_100,
      "nDCG@10": ndcg_cut_10,
  }

!mkdir -p eval_results
!python -m pyserini.eval.trec_eval -c -mrecall.100 -mndcg_cut.10 beir-v1.0.0-nfcorpus-test beir_embedding_k-ush/tevatron_dpr/rank.nfcorpus.trec > eval_results/dpr.result

before_trian_result = read_eval_result("eval_results/dpr.result")
pd.DataFrame([before_trian_result]).T

,0
R@100,0.1747
nDCG@10,0.1620


## DPRを訓練する

それでは、DPRを訓練していきます。

引き続きTevatronを使って、nfcorpusの訓練データセットでDPRを訓練していきますが、その前にnfcorpusの訓練データをTevatronで扱えるフォーマットに直す必要があります。

### 訓練データの整形
Tevatronでの訓練データセットの形は以下の形式のjsonで構成されるjsonlです。

内部的にはdatasetsのload_datasetを読んでいるので、load_datasetで読んで下記の構造のデータになればなんでも良いです。

```
{
  query_id: ...,
  query: ...,
  positive_passages: [
    {
      docid: ...,
      title: ...,
      text: ...,
    }
  ],
  negative_passages: [
    {
      docid: ...,
      title: ...,
      text: ...,
    }
  ]
}
```

### ir-datasets https://github.com/allenai/ir_datasets/

nfcorpusの訓練データを整形するわけですが、
その訓練データは ir-datasetsから引っ張ってきます。
ir-datasetsは情報検索向けのデータセットを集めて、同じようなインターフェースからアクセスできるようにしているライブラリです。

以下のような特徴があります。

- 多くの情報検索データセットを網羅
- 沢山のデータセットを共通化されたインターフェースからアクセスできる
- pip installだけで使い始めることができる
- 膨大なコーパスから高速な文書のlookupができるDocStoreなど、欲しい機能が揃っている

In [ ]:
!pip install ir_datasets

In [ ]:
from collections.abc import Callable
from pathlib import Path
from typing import Union, Dict, List, Set, Callable
from collections import defaultdict, namedtuple
import random
import json

import ir_datasets as irds
from tqdm import tqdm

Qrels = Dict[str, Dict[str, str]]

class IrdsToTevatronDataset(object):
  def __init__(self, dataset_key: str) -> None:
    self.dataset = self._load_dataset(dataset_key)

  def _load_dataset(self, dataset_key: str) -> irds.Dataset:
    return irds.load(dataset_key)

  def load_doc_ids(self) -> List[str]:
    doc_ids = []
    for doc in tqdm(self.dataset.docs_iter(), total=self.dataset.docs_count(), desc="loading doc id"):
      doc_ids.append(doc.doc_id)
    return doc_ids

  def load_query_table(self, query_field: str = "text") -> Dict[str, str]:
    queries = {}
    for query in tqdm(self.dataset.queries_iter(), total=self.dataset.queries_count(), desc="loading query"):
      queries[query.query_id] = getattr(query, query_field)
    return queries

  def load_qrels_table(self) -> Qrels:
    qrels = defaultdict(dict)
    for qrel in tqdm(self.dataset.qrels_iter(), total=self.dataset.qrels_count(), desc="loading qrels"):
      qrels[qrel.query_id][qrel.doc_id] = qrel.relevance
    return qrels

  def sample_random_negatives(self, doc_ids: List[str], exclude_doc_ids: Union[Set[str], List[str]], k: int = 1) -> List[str]:
    exclude_doc_ids = set(exclude_doc_ids)
    sample_source = [doc_id for doc_id in doc_ids if not doc_id in exclude_doc_ids]
    negative_doc_ids = random.choices(sample_source, k=k)
    return random.choices(sample_source, k=k)

  def prepare_train_dataset(self, output_path: Union[str, Path], doc_preprocess: Callable[[namedtuple], str], queries_num: int = 500):
    output_path = Path(output_path)

    docstore = self.dataset.docs_store()
    doc_ids = self.load_doc_ids()
    queries = self.load_query_table()
    qrels = self.load_qrels_table()

    with output_path.open("w") as fw:
      total = min(len(queries), queries_num)
      for i, qid in enumerate(tqdm(queries, desc="writing train dataset", total=total)):
        if i >= total: break

        query = queries[qid]
        relevant_doc_ids = [doc_id for doc_id in qrels[qid].keys() if qrels[qid][doc_id] > 0]
        negative_ids = self.sample_random_negatives(doc_ids, relevant_doc_ids, k=len(relevant_doc_ids))
        negatives = [doc_preprocess(doc) for docid, doc in docstore.get_many(negative_ids).items()]
        positives = [doc_preprocess(doc) for docid, doc in docstore.get_many(relevant_doc_ids).items()]
        train_json = {
            "query_id": qid,
            "query": query,
            "positive_passages": positives,
            "negative_passages": negatives
        }
        fw.write(json.dumps(train_json, ensure_ascii=False) + "\n")

dataset_key = "beir/nfcorpus/train"
dataset_path = "/content/nfcorpus.train.jsonl"
queries_num = 1000

def nfcorpus_doc_preprocess(doc: namedtuple) -> str:
  return {
      "docid": doc.doc_id,
      "title": doc.title,
      "text": doc.text,
  }

irds_to_tev = IrdsToTevatronDataset(dataset_key)
irds_to_tev.prepare_train_dataset(dataset_path, doc_preprocess=nfcorpus_doc_preprocess, queries_num=queries_num)

### **訓練の実行**

それでは先ほど評価したDPRを、nfcorpusの訓練データで訓練していきます。

訓練には`tevatron.driver.train`モジュールを用います。
内部的には、transformersのTrainerを使っており、カスタマイズもしやすくなっています。

今回は単純なDPRの訓練ですが、Tevatronではcross-encoderを教師とした蒸留や、spladeやunicoilなどの教師あり疎検索モデルの訓練も可能です。

主なパラメータの説明

| 名前 | 説明 |
| - | - |
| output_dir | モデルの出力先ディレクトリ |
| model_name_or_path | 学習対照のモデル |
| dataset_name | データセット、もしくはそのフォーマットの指定 |
| train_n_passages | 1 instanceの訓練で使うパッセージの数 |
| grad_cache | GradCacheというcross-batchに対照学習を行うライブラリを使うかどうか |


### 大規模言語モデルの訓練設定
大規模言語モデルに基づく検索モデルを訓練するときの方法やテクニックとしてよく用いられるものには以下のようなものがあります。
- 蒸留
  - cross encoderは密検索モデルや疎検索モデルより性能が高いですが、計算コストが大きいという課題がありました。
  - cross encoderの性能の高さを活かしつつ推論時の計算コストを小さくするために、cross encoderを教師として密検索モデルや疎検索モデルを訓練すると、性能が向上するということが知られています。
- 負例選択（negatives minitng）
  - in-batch negatives
    - 先ほどDPRの説明のところで紹介があった手法ですが、広くよく使われる設定です。
  - ANCE negatives
    - 学習中のモデルから負例を得る方法はhard negativesと呼ばれます。
    - やり方にもバリエーションがあり、ANCEのように学習が進むごとにnegativesを更新していく方法や、1度 in-batch negatives等で学習したモデルから負例を得て、得られた負例を使ってモデルをさらに学習させる等があります。
      - Zhang et al. Optimizing Dense Retrieval Model Training with Hard Negatives. SIGIR, 2021.
  - cross-batch negatives
    - in-batch negatives では、バッチサイズが大きくなるほど負例の数も大きくなります。一般に負例の数が大きくなると性能も高くなることが知られているので、in-batch negativesではなるべく譜例を大きくしたいです。
    - ですが、GPUメモリが十分に大きくないとバッチサイズは大きくできません。
    - これを克服する方法の一つとして、「他のバッチのデータを負例として使う」という方法があります。これがcross-batch negativesです。
    - TevatronはGradcacheというライブラリを学習に使っていますが、これはcross-batch negativesに分類されるような、負例選択手法です。
      - Gao et al. Scaling Deep Contrastive Learning Batch Size under Memory Limited Setup. RepL4NLP, 2021.
    - MoCoというcross-batch negatives手法とin-batch negativesでは性能の差が小さいため、多くの譜例を使えるcross-batchの方が有利という論文がある[2]が、32Kくらいバッチサイズを大きくするとin-batch negativesの方が性能が高いという論文[3]もあり、in-batchとcross-batchのどちらが良いかはまだはっきりしているとは言えない。
      - [2] Izacard et al. Contriever: Unsupervised Dense Information Retrieval with Contrastive Learning. TMLR, 2022.
      - [3] Wang et al. Text Embeddings by Weakly-Supervised Contrastive Pre-training. arXiv, 2022.


In [ ]:
%env TRAINED_DPR_DIR=dpr_nfcorpus
!CUDA_VISIBLE_DEVICES=0 python -m tevatron.driver.train \
  --output_dir $TRAINED_DPR_DIR \
  --model_name_or_path $BASE_IR_MODEL \
  --save_steps 10 \
  --dataset_name Tevatron/wikipedia-nq \
  --train_dir /content/nfcorpus.train.jsonl \
  --fp16 \
  --per_device_train_batch_size 128 \
  --positive_passage_no_shuffle \
  --train_n_passages 2 \
  --learning_rate 1e-5 \
  --q_max_len 32 \
  --p_max_len 156 \
  --num_train_epochs 5 \
  --logging_steps 500 \
  --grad_cache \
  --overwrite_output_dir

## 評価

それでは、訓練したDPRを評価してみましょう！

[最初に評価した時](#scrollTo=5M0VfI9pkyhH)と同じ nfcorpus のテストデータで評価します。
手順も同じです。

In [ ]:
!bash tevatron/scripts/eval_beir.sh dpr_nfcorpus nfcorpus

In [ ]:
!python -m pyserini.eval.trec_eval -c -mrecall.100 -mndcg_cut.10 beir-v1.0.0-nfcorpus-test beir_embedding_${TRAINED_DPR_DIR}/rank.nfcorpus.trec > eval_results/${TRAINED_DPR_DIR}.result
trained_dpr_result = read_eval_result("eval_results/dpr_nfcorpus.result")
pd.DataFrame([trained_dpr_result]).T

最後に、訓練前と後を同じ表にまとめて性能の変化を確認し、終了です。

In [ ]:
pd.DataFrame([before_trian_result, trained_dpr_result]).T